In [17]:
#To remove all variables from the namespace
%reset -f

#Creating a log file to record the commands and outputs
%logstop
%logstart -t -o "E:/Python Clinical Course/list2 log.txt"

Activating auto-logging. Current session state plus future input saved.
Filename       : E:/Python Clinical Course/list2 log.txt
Mode           : backup
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


In [23]:
import pandas as pd
import numpy as np
import pyreadstat
import os
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

In [24]:
adam_path = r"E:\Python Clinical Course\ADAM datasets\ADaM Datasets"

In [25]:
adam_datasets = {}

for file in os.listdir(adam_path):
    if file.endswith(".sas7bdat"):
        dataset_name = file.replace(".sas7bdat", "")
        file_path = os.path.join(adam_path, file)
        df, meta = pyreadstat.read_sas7bdat(file_path)
        adam_datasets[dataset_name] = df

In [26]:
# Load the data
adsl = adam_datasets.get("adsl")
adsl = adsl[['USUBJID', 'RFICDTC']]

In [27]:
# Step 2: Pagination variables (lnt and page1)
adsl = adsl.reset_index(drop=True)
adsl['lnt'] = (adsl.index % 20) + 1
adsl['page1'] = (adsl.index // 20) + 1


In [28]:

# Step 3: Create Word document
# Create the document
doc = Document()

# Titles
doc.add_paragraph('COVID-19 AA').alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
doc.add_paragraph('Protocol: 043').alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
title_para = doc.add_paragraph('16.2.1.2 Informed Consent')
title_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

# Iterate by pages
for page_num in sorted(adsl['page1'].unique()):
    sub_df = adsl[adsl['page1'] == page_num]
    
    table = doc.add_table(rows=1, cols=2)
    table.style = 'Light Shading Accent 1'

    # Set header
    hdr_cells = table.rows[0].cells
    widths = [Inches(6.0), Inches(6.5)] 
    hdr_cells[0].text = 'Subject Number'
    hdr_cells[1].text = 'Date of Informed Consent'
    for i, cell in enumerate(hdr_cells):
        cell.width = widths[i]

    # Fill rows
    for _, row in sub_df.iterrows():
        row_cells = table.add_row().cells
        widths = [Inches(6.0), Inches(6.5)] 
        row_cells[0].text = str(row['USUBJID'])
        row_cells[1].text = str(row['RFICDTC'])
        for i, cell in enumerate(row_cells):
            cell.width = widths[i]

    # Horizontal line separator (simulate compute block)
    #doc.add_paragraph('_________________________________________')

    # Page break after each group
    doc.add_page_break()

# --- FOOTNOTE ---
footer = doc.add_paragraph('E:\\LIS2.SAS')
footer.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT


In [29]:

# Step 5: Save document
doc.save(r"E:\Python Clinical Course\TLF\output\l_16_2_1_2.docx")
